In [1]:
# Zelle 1: (optional) Abhängigkeiten installieren — nur ausführen, falls nötig
# Führ diese Zelle nur einmal aus, wenn pandas oder ipywidgets fehlen.
import sys
!{sys.executable} -m pip install --quiet pandas ipywidgets || true

In [2]:
# Zelle 2: Imports und Pfade vorbereiten
import os
import json
from datetime import datetime, date
import pandas as pd
from IPython.display import display, HTML

# Optional: ipywidgets (falls installiert)
try:
    import ipywidgets as widgets
    WIDGETS_AVAILABLE = True
except Exception:
    WIDGETS_AVAILABLE = False

DATA_DIR = os.path.join(os.getcwd(), "campus_data")
os.makedirs(DATA_DIR, exist_ok=True)
DATA_FILE = os.path.join(DATA_DIR, "data.json")
SURVEY_FILE = os.path.join(DATA_DIR, "survey.json")

print("Datenverzeichnis:", DATA_DIR)
print("ipywidgets verfügbar:", WIDGETS_AVAILABLE)


Datenverzeichnis: /home/wih42zor/campusEingang/campus_data
ipywidgets verfügbar: True


In [3]:
# Zelle 2: Imports und Pfade vorbereiten
import os
import json
import tempfile
from datetime import datetime, date
from threading import Thread
import pandas as pd

# Datenverzeichnis im Notebook-Ordner (schreibbar) oder temporär
DATA_DIR = os.path.join(os.getcwd(), "campus_data")
os.makedirs(DATA_DIR, exist_ok=True)

DATA_FILE = os.path.join(DATA_DIR, "data.json")
SURVEY_FILE = os.path.join(DATA_DIR, "survey.json")

print("Datenverzeichnis:", DATA_DIR)

Datenverzeichnis: /home/wih42zor/campusEingang/campus_data


In [4]:
# Zelle 3: Beispiel-Daten, Lade-/Speicherfunktionen
default_data = {
    "tasks": [
        {"id": 1, "title": "Immatrikulation abschließen", "category": "Immatrikulation",
         "deadline": "2026-10-01", "link": "https://uni.example/immatrikulation", "done": False,
         "notes": "Unterlagen mitbringen"},
        {"id": 2, "title": "Chipkarte abholen", "category": "Organisatorisch",
         "deadline": "2024-01-01", "link": "", "done": False, "notes": ""}
    ],
    "next_id": 3
}

def ensure_files():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, "w", encoding="utf-8") as f:
            json.dump(default_data, f, ensure_ascii=False, indent=2)
    if not os.path.exists(SURVEY_FILE):
        with open(SURVEY_FILE, "w", encoding="utf-8") as f:
            json.dump([], f, ensure_ascii=False, indent=2)

def load_data(path=DATA_FILE):
    ensure_files()
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def save_data(data, path=DATA_FILE):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def load_survey(path=SURVEY_FILE):
    ensure_files()
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def save_survey(entries, path=SURVEY_FILE):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(entries, f, ensure_ascii=False, indent=2)

ensure_files()
print("Data files ready:", os.path.exists(DATA_FILE), os.path.exists(SURVEY_FILE))


Data files ready: True True


In [5]:
# Zelle 4: Hilfsfunktionen für Analyse (DataFrame, Overdue-Berechnung, Anzeige)
def tasks_to_df(data):
    df = pd.DataFrame(data["tasks"])
    if df.empty:
        df = pd.DataFrame(columns=["id","title","category","deadline","link","done","notes"])
    if "deadline" not in df.columns:
        df["deadline"] = ""
    def parse_deadline(s):
        try:
            return datetime.strptime(s, "%Y-%m-%d").date()
        except:
            return None
    df["deadline_date"] = df["deadline"].apply(parse_deadline)
    today = date.today()
    df["overdue"] = df.apply(lambda r: (not bool(r.get("done"))) and (r["deadline_date"] is not None) and (r["deadline_date"] < today), axis=1)
    df = df.sort_values(by=["overdue", "deadline_date"], ascending=[False, True])
    return df

def show_tasks_table(df):
    if df.empty:
        display(HTML("<p><em>Keine Aufgaben vorhanden.</em></p>"))
    else:
        display(df[["id","title","category","deadline","done","overdue","notes"]].reset_index(drop=True))

# Beispiel: lade und zeige aktuelle Tabelle
data = load_data()
df = tasks_to_df(data)
show_tasks_table(df)


,id,title,category,deadline,done,overdue,notes
0,2,Chipkarte abholen,Organisatorisch,2024-01-01,False,True,
1,1,Immatrikulation abschließen,Immatrikulation,2026-10-01,False,False,Unterlagen mitbringen
2,3,jdc,All,2026,False,False,jsv


In [6]:
# Zelle 5: CRUD-Funktionen (zum interaktiven Verwenden im Notebook)
def add_task(title, category="All", deadline="", link="", notes=""):
    data = load_data()
    nid = data.get("next_id", 1)
    task = {"id": nid, "title": title, "category": category, "deadline": deadline, "link": link, "notes": notes, "done": False}
    data["tasks"].append(task)
    data["next_id"] = nid + 1
    save_data(data)
    return task

def edit_task(task_id, **kwargs):
    data = load_data()
    task = next((t for t in data["tasks"] if t["id"]==task_id), None)
    if not task:
        raise ValueError("Task not found")
    for k,v in kwargs.items():
        if k in task:
            task[k] = v
    save_data(data)
    return task

def toggle_done(task_id):
    data = load_data()
    task = next((t for t in data["tasks"] if t["id"]==task_id), None)
    if not task:
        raise ValueError("Task not found")
    task["done"] = not task.get("done", False)
    save_data(data)
    return task

def delete_task(task_id):
    data = load_data()
    data["tasks"] = [t for t in data["tasks"] if t["id"]!=task_id]
    save_data(data)
    return True

# Beispiele (ausführen, um zu testen)
# add_task("Prüfungsanmeldung", "Prüfungen", "2026-09-15", "https://uni.example/pruefungen", "Anmeldeportal nutzen")
# toggle_done(1)
# edit_task(2, title="Chipkarte abholen (aktualisiert)")
# delete_task(99)


In [7]:
# Zelle 6: Nützliche Analyse-Queries (ausführen, um Ergebnisse zu sehen)
data = load_data()
df = tasks_to_df(data)

# 1) Alle überfälligen Aufgaben
df_overdue = df[df["overdue"]]
print("Überfällige Aufgaben:", len(df_overdue))
display(df_overdue[["id","title","category","deadline","notes"]])

# 2) Offene Aufgaben (nicht erledigt)
df_open = df[~df["done"]]
print("Offene Aufgaben insgesamt:", len(df_open))
display(df_open[["id","title","category","deadline","overdue"]])

# 3) Anzahl pro Kategorie
print("Anzahl Aufgaben pro Kategorie:")
display(df.groupby("category").size().sort_values(ascending=False).rename("count").to_frame())


Überfällige Aufgaben: 1


,id,title,category,deadline,notes
1,2,Chipkarte abholen,Organisatorisch,2024-01-01,


Offene Aufgaben insgesamt: 3


,id,title,category,deadline,overdue
1,2,Chipkarte abholen,Organisatorisch,2024-01-01,True
0,1,Immatrikulation abschließen,Immatrikulation,2026-10-01,False
2,3,jdc,All,2026,False


Anzahl Aufgaben pro Kategorie:


,count
category,
All,1
Immatrikulation,1
Organisatorisch,1


In [8]:
# Zelle 7: Optional interaktive UI mit ipywidgets (nur wenn ipywidgets installiert ist)
if WIDGETS_AVAILABLE:
    # Widgets definieren
    title_w = widgets.Text(description="Titel:", layout=widgets.Layout(width="60%"))
    category_w = widgets.Text(description="Kategorie:", value="All")
    deadline_w = widgets.Text(description="Frist (YYYY-MM-DD):", value="")
    link_w = widgets.Text(description="Link:", value="")
    notes_w = widgets.Textarea(description="Notizen:", layout=widgets.Layout(width="60%", height="80px"))
    add_btn = widgets.Button(description="Aufgabe hinzufügen", button_style="success")
    output = widgets.Output()

    def on_add_clicked(b):
        with output:
            try:
                task = add_task(title_w.value.strip() or "Neue Aufgabe", category_w.value.strip() or "All",
                                deadline_w.value.strip(), link_w.value.strip(), notes_w.value.strip())
                print("Hinzugefügt:", task["id"], task["title"])
                # clear inputs
                title_w.value = ""
                category_w.value = ""
                deadline_w.value = ""
                link_w.value = ""
                notes_w.value = ""
            except Exception as e:
                print("Fehler:", e)
            # show updated table
            data = load_data()
            df = tasks_to_df(data)
            display(df[["id","title","category","deadline","done","overdue"]])

    add_btn.on_click(on_add_clicked)

    # Anzeige-Layout
    form = widgets.VBox([title_w, category_w, deadline_w, link_w, notes_w, add_btn, output])
    display(form)
else:
    print("ipywidgets nicht installiert. Installiere mit: pip install ipywidgets")


In [9]:
# Zelle 8: Umfrage-Funktionen (Pain Points) und Anzeige
def add_survey_entry(name, problem, urgency="Mittel"):
    entries = load_survey()
    entry = {"name": name.strip(), "problem": problem.strip(), "urgency": urgency, "time": datetime.now().strftime("%Y-%m-%d %H:%M")}
    entries.insert(0, entry)
    save_survey(entries)
    return entry

def show_survey():
    entries = load_survey()
    if not entries:
        display(HTML("<p><em>Noch keine Umfrageeinträge.</em></p>"))
    else:
        html = "<ul>"
        for e in entries:
            html += f"<li><strong>{e.get('name') or 'Anonym'}</strong> ({e.get('urgency')}) — {e.get('problem')} <span style='color:#666;font-size:0.9em'>[{e.get('time')}]</span></li>"
        html += "</ul>"
        display(HTML(html))

# Beispiel: add_survey_entry("Max", "Unklare Prüfungsanmeldung", "Hoch")
# show_survey()
